This script is meant to take the dataframe created in TrackingHandsFromVideoWritingtoCSV and explicitly link together occurrences of the same hand, across frames, as well as to try to link together two hands belonging to the same person.
This is necessary because otherwise we do not have a time-series, but rather a series of individual frames, each with a variable number of tracked hands. 
The first block therefore loads the required packages and defines some functions that we will use.

In [ ]:
import pandas as pd
import numpy as np
from os import listdir
from os.path import isfile, join


#list all videos in mediafolder
mypath = "./MediaToAnalyze/"
onlyfiles = [f for f in listdir(mypath) if isfile(join(mypath, f))] # get all files that are in mediatoanalyze
#time series output folder
foldtime = "./Timeseries_Output/"

def get_closest_hand(hand,prev_frame, detected_this_frame):
    # this function looks at the hand coordinates given for the current frame, and tries to find the closest match in the previous frame
    # The idea being that, from one frame to the next, the same hand should appear in only a slightly different position
    # Note that this may not be the case when tracking is lost, but the hand moves some distance away.
    tolerance = 3 # somewhat arbitrary number, but ensures that we don't take hands that are extremely far away
    distance_list = []
    
    # first we drop any hands that have already been connected to one this frame
    prev_frame = prev_frame[~prev_frame.hand_ID.isin(detected_this_frame)]
    # then we need to reset the index (due to the prev_frame being composed of rows from different
    # points in the dataframe)
    prev_frame = prev_frame.reset_index()
    # one hand at a time..
    for _,prev_hand in prev_frame.iterrows():
        #.. we compare coordinates between current hand and previous hands
        distance = 0
        for name, vals in prev_hand.iteritems():
            if name[0] == 'X' or name[0] == 'Y' or name[0] == 'Z':
                distance += np.abs(prev_hand[name] - hand[name])
        # take the sum of all coordinate differences
        distance_list.append(np.sum(distance))
    # find the closest match
    closest = np.min(distance_list)
    closest_ID = prev_frame["hand_ID"][np.where(distance_list == np.min(distance_list))[0][0]]
    # make sure it is within some threshold
    if closest < tolerance:
        return closest_ID
    else:
        return np.nan
  
    
def getslope(a,b):
    # simple function to calculate the slope based on two points
    slope = (b[1]-a[1]) / (b[0]-a[0])
    return slope

def get_intercept(slope, x,y):
    # gets the y-intercept
    intercept = y-slope*x
    return intercept


def get_interceptX(slope, p1,p2):
    # get the x-intercept
    a = p1[1] - p2[1]
    b = p1[0] - p2[0]
    
    m = a/b
    c = p1[1]-m*p1[0]
    # solve for y
    y = 0
    x_intercept = (y-c)/m
    return x_intercept

def output_progress(df_idx, len_df, checkpoints):
    # This funcion is just to give us some progress update, since the script can take a minute to run. This way we know it hasn't crashed.
    if df_idx >= len_df and 1 in checkpoints:
        checkpoints.remove(1)
        print("done!")
    elif df_idx > len_df*0.8 and 0.8 in checkpoints:
        checkpoints.remove(0.8)
        print("80% complete")
    elif  df_idx > len_df*0.6 and 0.6 in checkpoints:
        checkpoints.remove(0.6)
        print("60% complete")
    elif  df_idx > len_df*0.4 and 0.4 in checkpoints:
        checkpoints.remove(0.4)
        print("40% complete")
    elif  df_idx > len_df*0.2 and 0.2 in checkpoints:
        checkpoints.remove(0.2)
        print("20% complete")
        
def flip(original_val):
    trans_val = 1 - original_val
    return trans_val
        
     

In [ ]:
# This block goes through the entire dataframe, frame by frame, and assigns an ID to each uniquely tracked hand, then tries to find pairs of hands

# In this first block of the script, we go through each file, and try to track hands from frame to frame. 
# While we're going through the files, a block within this set of loops will determine the orientation of each hand with respect
# to the sides of the screen. This is used in the block directly below (in this same cell of the notebook) to try to find pairs of hands.

datafiles = os.listdir(foldtime)
for datafile in datafiles:
    
    print("working on " + datafile + "...")
    
    df = pd.read_csv(foldtime + datafile)
    # create empty column for the hand IDs
    df["hand_ID"] = np.nan
    df_idx = 0
    arbID = 0 # this will be a first pass ID 
    
    # a little progress counter so we know how far along it is
    df_len = len(df)
    checkpoints = [0.2,0.4,0.6,0.8]
    
    while df_idx < len(df):
        
        
        current_time = df["time"][df_idx]
        frame = df.loc[df["time"] == current_time]
        
        detected_this_frame = [] # we set this up to not let two hands be assigned to the same previous hand
        for frame_idx,hand in frame.iterrows():
            # if this is the first frame, each Hand is new, and receives a new ID
            if df_idx == 0:
                df.loc[frame_idx,"hand_ID"] = arbID
                arbID +=1
            else:
                
            # after the first frame, we need to try to match each hand with a previous ID
                closest_ID = get_closest_hand(hand,prev_hands, detected_this_frame)
                
                detected_this_frame.append(closest_ID)
            
                # if there are no good matches, we assume it's a new (previously untracked) hand
                # and we assign it a new ID
                if np.isnan(closest_ID):
                    closest_ID = arbID
                    arbID +=1
                
                df.loc[frame_idx,"hand_ID"] = closest_ID
                
                ##################################
                #### This block calculates hand orientation ####
                # so first get middle point of base
                midbase = [hand["X_PINKY_MCP"] + ((hand["X_INDEX_MCP"] - hand["X_PINKY_MCP"])/2),
                           flip(hand["Y_PINKY_MCP"]) + ((flip(hand["Y_INDEX_MCP"]) - flip(hand["Y_PINKY_MCP"]))/2)]
                wrist_coords = [hand["X_WRIST"],flip(hand["Y_WRIST"])]
                if midbase[0] > hand["X_WRIST"]:
                    hand_direction = "R"
                else:
                    hand_direction = "L"
                    # if left facing, we need to shift the coordinates to the other side of the y-axis
                    wrist_coords[0] = hand["X_WRIST"]*-1
                    midbase[0] = midbase[0]*-1
                
                # then the slope from mid base to tip
                slope = getslope([hand["X_WRIST"],flip(hand["Y_WRIST"])],midbase)
                
                # now get the initial X and Y intercept
                # this is used to figured out where in our image the hand seems 
                # to be projecting from (Top, Bottom, Left or Right)
                intercept_X = get_interceptX(slope,
                                               wrist_coords,
                                               midbase) 
                intercept_Y = get_intercept(slope,wrist_coords[0],wrist_coords[1]) 
                
                if hand_direction == "R" and intercept_Y < 0 and 0 < intercept_X < 1:
                    origin = "Bot"
                elif hand_direction == "R" and 0 < intercept_Y < 1:
                    origin = "Left"
                elif hand_direction == "R" and intercept_Y > 1 and intercept_X > 0:
                    origin = "Top"
                    intercept_X = get_interceptX(slope,
                                               [wrist_coords[0],wrist_coords[1]*-1],
                                               midbase) 
                elif hand_direction =="L" and intercept_Y < 0 and intercept_X > -1:
                    origin = "Bot"
                    intercept_X = intercept_X*-1
                elif hand_direction =="L" and 0 < intercept_Y < 1:
                    origin = "Right"
                elif hand_direction =="L" and intercept_Y > 1 and intercept_X < 0:
                    origin = "Top"
                    intercept_X = get_interceptX(slope,
                                               [wrist_coords[0],wrist_coords[1]*-1],
                                               midbase) 
                
                    
                # update this info
                df.loc[frame_idx,"origin"] = origin
                df.loc[frame_idx,"x_intercept"] = intercept_X
                df.loc[frame_idx,"y_intercept"] = intercept_Y
                ##################################
                
        # this needs to be updated to reflect the new hand_IDs
        frame = df.loc[df["time"] == current_time]
        # if this is the first frame, we store these values for later
        if df_idx == 0:        
            prev_hands = frame.copy()
        # after getting this first set of hand-coordinates, we need to update it on each frame
        else:
            # for each hand in the current frame, check..
            for _,hand in frame.iterrows():
                # .. if it's been logged in prev_hands already, update it
                if hand["hand_ID"] in prev_hands["hand_ID"]:
                    prev_hands.loc[hand["hand_ID"]] = hand
                else:
                    prev_hands = prev_hands.append(hand)
                
        #move the index forward to the next time point
        df_idx += len(frame)
        
        # check progress
        output_progress(df_idx,df_len, checkpoints)

                
    df_idx = 0
    while df_idx < len(df):
        
        
        current_time = df["time"][df_idx]
        frame = df.loc[df["time"] == current_time]
        # create a new dataframe where collect all of the right-hand distances from this left
        pairing_df = pd.DataFrame()
        pairing_idx = 0
        

        
        for frame_idx,hand_origin in frame.iterrows():            
            # Once we collect both intercepts (each will have one zero in it), we need to pair with another hand
            # so first we add an identifier indicating wall orientation (left, right, top, bot)
            # For left- or right-oriented, we assume that a right hand should pair with a left hand with a
            #    higher y-value, with the same LR orientation
            # For top or bot oriented, we assume L should pair with R with a lower (top) or higher (bot)
            #   x-value, with the same orientation
            ## based on intercepts
            if hand_origin["hand"][10] == "L":
                
                # once we find a left hand, we need to find any potential match
                for match_idx,hand in frame.iterrows():
                    # for each potential match in the frame, we need to record the ID of the left hand (origin hand),
                    # as well as the distance and ID of the potential pair
                    # That way, at the end we can sort out which ones fit best together
                    if hand["hand"][10] == "R":
                        
                        # if oriented bottom and we slide from left to right
                        if hand_origin["origin"] == "Bot":
                            # first we check if there is a right hand, also with an origin on the bottom
                            # and that is to the right of the left hand we are looking at
                            if (hand["x_intercept"] - hand_origin["x_intercept"] > 0) and hand["origin"] == "Bot":
                                # we want this to be >0, because that indicates
                                # that the right hand is indeed to the right of the left
                                pairing_df.loc[pairing_idx,"origin_ID"] = hand_origin["hand_ID"]
                                pairing_df.loc[pairing_idx,"hand_id"] = hand["hand_ID"]
                                pairing_df.loc[pairing_idx,"pairing_dist"] = hand["x_intercept"] - hand_origin["x_intercept"]
                                pairing_df.loc[pairing_idx,"idx"] = frame_idx 
                                pairing_df.loc[pairing_idx,"match_idx"] = match_idx 
                                pairing_idx+=1
                            #if not, we check if this right hand is on the Right side
                            elif  hand["origin"] == "Right":
                                distance = hand["y_intercept"] + (1 - hand_origin["x_intercept"])
                                # distance < 1 is an arbitrary cutoff just to ensure it's not picking up
                                # a hand on the other side of the screen
                                if distance > 0 and distance < 1:
                                    pairing_df.loc[pairing_idx,"origin_ID"] = hand_origin["hand_ID"]
                                    pairing_df.loc[pairing_idx,"hand_id"] = hand["hand_ID"]
                                    pairing_df.loc[pairing_idx,"pairing_dist"] = hand["y_intercept"] + (hand["x_intercept"] - hand_origin["x_intercept"])
                                    pairing_df.loc[pairing_idx,"idx"] = frame_idx
                                    pairing_df.loc[pairing_idx,"match_idx"] = match_idx 
                                    pairing_idx+=1
                        # if oriented top, we go right to left
                        elif  hand_origin["origin"] == "Top":
                                #is this right hand also on the top?
                            if hand["origin"] == "Top" and (hand_origin["x_intercept"] - hand["x_intercept"] >0):
                                    pairing_df.loc[pairing_idx,"origin_ID"] = hand_origin["hand_ID"]
                                    pairing_df.loc[pairing_idx,"hand_id"] = hand["hand_ID"]
                                    pairing_df.loc[pairing_idx,"pairing_dist"] = hand_origin["x_intercept"] - hand["x_intercept"]
                                    pairing_df.loc[pairing_idx,"idx"] = frame_idx
                                    pairing_df.loc[pairing_idx,"match_idx"] = match_idx 
                                    pairing_idx+=1
                           # slide up the next side, if there is anything there
                            elif hand["origin"] == "Left":
                                distance =(1 -hand["y_intercept"]) +(1 - hand_origin["x_intercept"])
                                # distance < 1 is an arbitrary cutoff just to ensure it's not picking up
                                # a hand on the other side of the screen
                                if distance > 0 and distance < 1:
                                    pairing_df.loc[pairing_idx,"origin_ID"] = hand_origin["hand_ID"]
                                    pairing_df.loc[pairing_idx,"hand_id"] = hand["hand_ID"]
                                    pairing_df.loc[pairing_idx,"pairing_dist"] = hand["y_intercept"] +(hand_origin["x_intercept"] - hand["x_intercept"])
                                    pairing_df.loc[pairing_idx,"idx"] = frame_idx
                                    pairing_df.loc[pairing_idx,"match_idx"] = match_idx 
                                    pairing_idx+=1
                                    
                    
                                    
        # at the end of each frame, we need to pick the pairs
        while len(pairing_df) >0:
            best_match = pairing_df[["pairing_dist"]].idxmin()[0]
            match_origin= pairing_df.loc[best_match,"origin_ID"]
            match_pair = pairing_df.loc[best_match,"hand_id"]
            origin_idx = pairing_df.loc[best_match,"idx"]
            match_idx = pairing_df.loc[best_match,"match_idx"]
            # need to get row number (above)
            
            df.loc[origin_idx,"paired_hand"] =  match_pair
            df.loc[origin_idx,"pairing_distance"] =  pairing_df.loc[best_match,"pairing_dist"]
            df.loc[match_idx,"paired_hand"] = match_origin
            df.loc[match_idx,"pairing_distance"] =  pairing_df.loc[best_match,"pairing_dist"]

            # then remove these hands from the pairing_df
            pairing_df = pairing_df[pairing_df.origin_ID != match_origin]
            pairing_df = pairing_df[pairing_df.hand_id != match_pair]

           

        df_idx+= len(frame)

    df.to_csv(foldtime + datafile.split(".csv")[0] + "_paired.csv")